## Total Energy Burned
Active energy burned and basal energy burned

In [26]:
import pandas as pd

# Load file into dataframe
folder_path = 'c:/Users/aoife/Documents/Project/DataTables/'

df_aeb = pd.read_csv(folder_path + 'active-energy-burned.csv', usecols=['participantId', 'timestamp', 'startTime', 'endTime', 'value'], parse_dates=['timestamp', 'startTime', 'endTime'])
df_beb = pd.read_csv(folder_path + 'basal-energy-burned.csv', usecols=['participantId', 'timestamp', 'startTime', 'endTime', 'value'], parse_dates=['timestamp', 'startTime', 'endTime'])
df_bm = pd.read_csv(folder_path + 'body-mass.csv', usecols=['participantId', 'timestamp', 'value'], parse_dates=['timestamp'])
df_h = pd.read_csv(folder_path + 'height.csv', usecols=['participantId', 'timestamp', 'value'], parse_dates=['timestamp'])

print(df_aeb.head())
print('----------------\n')
print(df_beb.head())

# Create a list of dataframes

energy_dfs = [df_aeb, df_beb]
bmi_dfs = [df_bm, df_h]

                          participantId                 timestamp  \
0  b2571643-4aec-492f-bc7f-6f23c7fe239a 2018-10-03 12:45:22+00:00   
1  b2571643-4aec-492f-bc7f-6f23c7fe239a 2018-10-03 12:45:22+00:00   
2  b2571643-4aec-492f-bc7f-6f23c7fe239a 2018-10-03 12:45:22+00:00   
3  b2571643-4aec-492f-bc7f-6f23c7fe239a 2018-10-03 12:45:22+00:00   
4  b2571643-4aec-492f-bc7f-6f23c7fe239a 2018-10-03 12:45:22+00:00   

                  startTime                   endTime  value  
0 2018-10-03 10:44:19+00:00 2018-10-03 10:44:40+00:00  1.822  
1 2018-10-03 10:44:40+00:00 2018-10-03 10:45:41+00:00  0.711  
2 2018-10-03 10:45:41+00:00 2018-10-03 10:46:53+00:00  2.267  
3 2018-10-03 10:46:53+00:00 2018-10-03 10:47:54+00:00  2.569  
4 2018-10-03 10:47:54+00:00 2018-10-03 10:48:25+00:00  0.564  
----------------

                          participantId                 timestamp  \
0  b2571643-4aec-492f-bc7f-6f23c7fe239a 2018-10-03 12:45:23+00:00   
1  b2571643-4aec-492f-bc7f-6f23c7fe239a 2018-10-03 

In [27]:
# Create a new column which contains the difference between the start and end time

for df in energy_dfs:
    if 'startTime' and 'endTime' in df.columns:
        df['duration'] = df['endTime'] - df['startTime']



'''print(df_aeb.head())
print('----------------\n')
print(df_beb.head())'''

"print(df_aeb.head())\nprint('----------------\n')\nprint(df_beb.head())"

In [28]:
# Adjust the startTime to contain just the date

for df in energy_dfs:
    if 'startTime' in df.columns:
        df['startTime'] = pd.to_datetime(df['startTime'], format='%Y-%m-%dT%H:%M:%SZ')

        df['startDate'] = df['startTime'].dt.date

        print(df.head())
        print('----------------\n')

'''df_aeb['startTime'] = pd.to_datetime(df_aeb['startTime'], format='%Y-%m-%dT%H:%M:%SZ')

df_aeb['startDate'] = df_aeb['startTime'].dt.date

print(df_aeb.head())'''

                          participantId                 timestamp  \
0  b2571643-4aec-492f-bc7f-6f23c7fe239a 2018-10-03 12:45:22+00:00   
1  b2571643-4aec-492f-bc7f-6f23c7fe239a 2018-10-03 12:45:22+00:00   
2  b2571643-4aec-492f-bc7f-6f23c7fe239a 2018-10-03 12:45:22+00:00   
3  b2571643-4aec-492f-bc7f-6f23c7fe239a 2018-10-03 12:45:22+00:00   
4  b2571643-4aec-492f-bc7f-6f23c7fe239a 2018-10-03 12:45:22+00:00   

                  startTime                   endTime  value        duration  \
0 2018-10-03 10:44:19+00:00 2018-10-03 10:44:40+00:00  1.822 0 days 00:00:21   
1 2018-10-03 10:44:40+00:00 2018-10-03 10:45:41+00:00  0.711 0 days 00:01:01   
2 2018-10-03 10:45:41+00:00 2018-10-03 10:46:53+00:00  2.267 0 days 00:01:12   
3 2018-10-03 10:46:53+00:00 2018-10-03 10:47:54+00:00  2.569 0 days 00:01:01   
4 2018-10-03 10:47:54+00:00 2018-10-03 10:48:25+00:00  0.564 0 days 00:00:31   

    startDate  
0  2018-10-03  
1  2018-10-03  
2  2018-10-03  
3  2018-10-03  
4  2018-10-03  
--------

"df_aeb['startTime'] = pd.to_datetime(df_aeb['startTime'], format='%Y-%m-%dT%H:%M:%SZ')\n\ndf_aeb['startDate'] = df_aeb['startTime'].dt.date\n\nprint(df_aeb.head())"

In [29]:
# Get the total duration per day for each participant

for df in energy_dfs:
    if 'duration' in df.columns:
        df['duration'] = df.groupby(['participantId', 'startDate'])['duration'].transform('sum')
        df['value'] = df.groupby(['participantId', 'startDate'])['value'].transform('sum')

        # Drop duplicates
        df.drop_duplicates(subset=['participantId', 'startDate'], inplace=True)

        print(df.head())
        #print(df['value'].head())
        print('----------------\n')

'''df_aeb['duration'] = df_aeb.groupby(['participantId', 'startDate'])['duration'].transform('sum')
df_aeb['value'] = df_aeb.groupby(['participantId', 'startDate'])['value'].transform('sum')

print(df_aeb['duration'].head())'''

                            participantId                 timestamp  \
0    b2571643-4aec-492f-bc7f-6f23c7fe239a 2018-10-03 12:45:22+00:00   
137  be073dc4-f492-4cf3-a183-31cc15657a3f 2017-09-06 12:52:29+00:00   
180  be073dc4-f492-4cf3-a183-31cc15657a3f 2017-09-06 12:52:29+00:00   
435  be073dc4-f492-4cf3-a183-31cc15657a3f 2017-09-07 19:39:12+00:00   
590  be073dc4-f492-4cf3-a183-31cc15657a3f 2017-09-14 15:16:00+00:00   

                    startTime                   endTime    value  \
0   2018-10-03 10:44:19+00:00 2018-10-03 10:44:40+00:00  112.882   
137 2017-09-05 23:15:40+00:00 2017-09-05 23:16:42+00:00  130.400   
180 2017-09-06 00:00:06+00:00 2017-09-06 00:00:26+00:00  195.420   
435 2017-09-07 15:48:13+00:00 2017-09-07 15:49:14+00:00   78.150   
590 2017-09-14 13:40:24+00:00 2017-09-14 13:41:27+00:00  128.663   

           duration   startDate  
0   0 days 02:15:27  2018-10-03  
137 0 days 01:18:16  2017-09-05  
180 0 days 03:24:02  2017-09-06  
435 0 days 02:26:26  2017-09

"df_aeb['duration'] = df_aeb.groupby(['participantId', 'startDate'])['duration'].transform('sum')\ndf_aeb['value'] = df_aeb.groupby(['participantId', 'startDate'])['value'].transform('sum')\n\nprint(df_aeb['duration'].head())"

In [30]:
# Calculate total energy burned and total duration per day

# Sort by participantId
df_aeb.sort_values(by=['participantId'], inplace=True)
df_beb.sort_values(by=['participantId'], inplace=True)

# Rename value columns
df_aeb.rename(columns={'value': 'active_energy'}, inplace=True)
df_beb.rename(columns={'value': 'basal_energy'}, inplace=True)

# Merge on participantId and startDate

df_energy = pd.merge(df_aeb, df_beb, on=['participantId', 'startDate'])

# Create a new column which contains the total energy burned per day

df_energy['total_energy'] = df_energy['active_energy'] + df_energy['basal_energy']

# Create a new column which contains the total duration per day

df_energy['total_duration'] = df_energy['duration_x'] + df_energy['duration_y']

# Drop unnecessary columns

df_energy.drop(columns=['timestamp_x', 'startTime_x', 'endTime_x', 'timestamp_y', 'startTime_y', 'endTime_y'], inplace=True)

print(df_energy.head())
print('----------------\n')
print(df_energy['total_duration'].head())



                          participantId  active_energy      duration_x  \
0  00fd4039-9b5e-4bbb-8295-4983a3f58371         13.815 0 days 01:26:33   
1  017702d7-e540-4e1a-ab19-ca5f1c34e70b         41.148 0 days 02:01:34   
2  01866570-f91f-4d3b-9d41-0a8cdcbfa922         32.569 0 days 02:29:05   
3  0292a3a2-5d0b-42a7-ba86-03eae9cda7d5         13.806 0 days 00:15:01   
4  048ba877-5df6-47c4-b485-d3538db9e9b4         30.735 0 days 02:14:05   

    startDate  basal_energy      duration_y  total_energy  total_duration  
0  2018-08-15       227.847 0 days 04:01:00       241.662 0 days 05:27:33  
1  2017-12-23       879.016 0 days 12:30:39       920.164 0 days 14:32:13  
2  2018-02-25       685.827 0 days 08:29:49       718.396 0 days 10:58:54  
3  2017-11-24       933.963 0 days 10:28:11       947.769 0 days 10:43:12  
4  2017-11-17       593.295 0 days 08:14:35       624.030 0 days 10:28:40  
----------------

0   0 days 05:27:33
1   0 days 14:32:13
2   0 days 10:58:54
3   0 days 10:43:12
4

## BMI

Body Mass and Height

In [31]:
# Use the groupby function to get the average height/body mass for each participant

for df in bmi_dfs:
    if 'value' in df.columns:
        df['value'] = df.groupby(['participantId'])['value'].transform('mean')

        # Drop duplicates
        df.drop_duplicates(subset=['participantId'], inplace=True)

        print(df.head())
        print('----------------\n')

                          participantId                 timestamp       value
0  11b2bf4d-6020-4a86-81bd-237c5616c649 2016-08-24 23:56:14+00:00   97.976000
1  9cbbb597-30c0-4f48-aebd-7fe58c627cf6 2016-03-16 02:33:20+00:00   63.503000
2  941e9fa4-6ba5-41f5-9bcc-805849d751b8 2017-01-01 14:07:00+00:00   96.666000
3  41c683b0-f6c4-4a6a-8dd2-4b9a3dd30a32 2016-03-30 02:24:59+00:00  112.846333
4  35fbae45-0f21-45ea-8ed7-8d3361a29b86 2016-07-18 17:45:38+00:00   68.690083
----------------

                          participantId                 timestamp   value
0  11b2bf4d-6020-4a86-81bd-237c5616c649 2016-08-24 23:56:14+00:00  1.6256
1  9cbbb597-30c0-4f48-aebd-7fe58c627cf6 2016-03-16 02:33:20+00:00  1.6510
2  941e9fa4-6ba5-41f5-9bcc-805849d751b8 2017-01-01 14:07:00+00:00  1.8288
3  1cba6631-3fb8-4c18-904f-ab56fe82833d 2016-03-07 05:54:00+00:00  1.5494
4  eb2c4ff0-54e2-4c34-b73d-003e3e0427ad 2016-04-02 07:17:47+00:00  1.7018
----------------



In [32]:
# Calculate the BMI for each participant and create a new df

# Sort by participantId
df_bm.sort_values(by=['participantId'], inplace=True)
df_h.sort_values(by=['participantId'], inplace=True)

# Rename value columns
df_bm.rename(columns={'value': 'bodyMass'}, inplace=True)
df_h.rename(columns={'value': 'height'}, inplace=True)

# Merge on participantId

df_bmi = pd.merge(df_bm, df_h, on='participantId')

df_bmi['bmi'] = df_bmi['bodyMass'] / (df_bmi['height'] ** 2)

print(df_bmi.head())


                          participantId               timestamp_x  bodyMass  \
0  00a55fb5-da33-4e2e-ae61-28f589fcc174 2016-05-12 19:15:44+00:00    51.256   
1  00d6d2ee-ccea-45c7-9772-b19fd9bef2bf 2018-04-20 04:08:33+00:00    74.389   
2  00fd4039-9b5e-4bbb-8295-4983a3f58371 2018-08-15 04:32:59+00:00    59.874   
3  0113e483-0fc8-4892-a4fc-0b2f3820dde3 2019-03-06 19:58:35+00:00    68.039   
4  013d82d7-b3cc-4007-b00c-4d1d75bab9dd 2018-02-21 02:13:48+00:00    76.204   

                timestamp_y  height        bmi  
0 2016-05-12 19:15:44+00:00  1.5748  20.667783  
1 2018-04-20 04:08:33+00:00  1.9050  20.498343  
2 2018-08-15 04:32:59+00:00  1.6510  21.965653  
3 2019-03-06 19:58:35+00:00  1.7018  23.493130  
4 2018-02-21 02:13:48+00:00  1.6002  29.759747  


## Heart Rate

In [33]:

df_hr = pd.read_csv(folder_path + 'heart-rate.csv', usecols=['participantId', 'timestamp', 'startTime', 'endTime', 'value'], parse_dates=['timestamp', 'startTime', 'endTime'])

# Convert value from seconds to minutes

# df_hr['value'] = df_hr['value'] * 60

In [34]:
# Convert startTime to datetime and extract date

df_hr['startTime'] = pd.to_datetime(df_hr['startTime'], format='%Y-%m-%dT%H:%M:%SZ')
df_hr['endTime'] = pd.to_datetime(df_hr['endTime'], format='%Y-%m-%dT%H:%M:%SZ')
df_hr['startDate'] = df_hr['startTime'].dt.date
print(df_hr.head()) 

                          participantId                 timestamp  \
0  06bc6ebb-a233-469f-8091-90256f656b1b 2016-03-13 19:25:18+00:00   
1  06bc6ebb-a233-469f-8091-90256f656b1b 2016-03-13 19:25:18+00:00   
2  06bc6ebb-a233-469f-8091-90256f656b1b 2016-03-13 19:25:18+00:00   
3  06bc6ebb-a233-469f-8091-90256f656b1b 2016-03-13 19:25:18+00:00   
4  06bc6ebb-a233-469f-8091-90256f656b1b 2016-03-13 19:25:18+00:00   

                  startTime                   endTime  value   startDate  
0 2016-03-13 13:52:58+00:00 2016-03-13 13:52:58+00:00  1.500  2016-03-13  
1 2016-03-13 13:53:00+00:00 2016-03-13 13:53:00+00:00  1.500  2016-03-13  
2 2016-03-13 13:53:01+00:00 2016-03-13 13:53:01+00:00  1.517  2016-03-13  
3 2016-03-13 13:53:02+00:00 2016-03-13 13:53:02+00:00  1.533  2016-03-13  
4 2016-03-13 13:53:03+00:00 2016-03-13 13:53:03+00:00  1.550  2016-03-13  


In [37]:
# Get the total duration per day for each participant
# Need to consider that the start and end times are the same for all rows
# Figure out how to add the duration based off the end time of the previous row and the start time of the current row
# However, if the start time of the current row is larger than the end time of the previous row, do not add the duration, begin a new duration

from datetime import timedelta

# Function to calculate duration
def calculate_duration(group):
    group['prev_endTime'] = group['endTime'].shift(fill_value=None)
    group['duration'] = 0

    mask = (group['startTime'] - group['prev_endTime'] == timedelta(seconds=1))
    group.loc[mask, 'duration'] = 1

    # If you want to skip rows where startTime is more than 1 second after the previous endTime
    # mask = (group['startTime'] - group['prev_endTime'] > timedelta(seconds=1))
    # group = group.loc[~mask]

    group['duration'] = group['duration'].cumsum()

    return group

# Apply the function to each group (grouped by participantId and startDate)
df_hr = df_hr.groupby(['participantId', 'startDate'], group_keys=False).apply(calculate_duration)

# Calculate the total duration per person per day
daily_duration = df_hr.groupby(['participantId', 'startDate'])['duration'].max().reset_index()

# Merge the calculated durations back into the original dataframe
df_hr = pd.merge(df_hr, daily_duration, on=['participantId', 'startDate'], how='left', suffixes=('', '_daily'))

print(df_hr.head())






                          participantId                 timestamp  \
0  06bc6ebb-a233-469f-8091-90256f656b1b 2016-03-13 19:25:18+00:00   
1  06bc6ebb-a233-469f-8091-90256f656b1b 2016-03-13 19:25:18+00:00   
2  06bc6ebb-a233-469f-8091-90256f656b1b 2016-03-13 19:25:18+00:00   
3  06bc6ebb-a233-469f-8091-90256f656b1b 2016-03-13 19:25:18+00:00   
4  06bc6ebb-a233-469f-8091-90256f656b1b 2016-03-13 19:25:18+00:00   

                  startTime                   endTime  value   startDate  \
0 2016-03-13 13:52:58+00:00 2016-03-13 13:52:58+00:00  1.500  2016-03-13   
1 2016-03-13 13:53:00+00:00 2016-03-13 13:53:00+00:00  1.500  2016-03-13   
2 2016-03-13 13:53:01+00:00 2016-03-13 13:53:01+00:00  1.517  2016-03-13   
3 2016-03-13 13:53:02+00:00 2016-03-13 13:53:02+00:00  1.533  2016-03-13   
4 2016-03-13 13:53:03+00:00 2016-03-13 13:53:03+00:00  1.550  2016-03-13   

               prev_endTime  duration  duration_daily  duration_daily  
0                       NaT         0             350   

In [36]:
# Get the average heart rate per day for each participant

